In [1]:
# !pip install tweepy

In [2]:
# !pip show tweepy

In [3]:
import re
import socket
import tweepy

from tweepy import OAuthHandler
from tweepy import Stream

In [4]:
# Robert's credentials
ACCESS_TOKEN = "990614422497378305-cLwHkWduKtMDfvgfe5ksHUeFVbqurDH"
ACCESS_SECRET = "6cO51mZsOjPZ7ok96MFR1Ffbn7NtIG1zM7G6IVlafY99k"
CONSUMER_KEY = "wEmPdS3WOSd5tDFnWuTv8jam7"
CONSUMER_SECRET = "kD2cOFMVk1nO7Mfx0OJfkV4kVRAtoKDILhKphYDx3J90b69eLZ"
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAANsOigEAAAAAkCkfV5h6nt68GfoH%2Fw2yVOu%" + \
               "2BFbk%3DOGvzlHmFhgA8N7xYgj4f5c963UFW8qCZeo7HmvqMqFX6hEVlHD"

In [5]:
# the tags to track
add_tags = ["#worldcup2022"]
remove_tags = []

In [6]:
class MyStream(tweepy.StreamingClient):
    global client_socket

    def on_tweet(self, tweet):
        try:
            msg = tweet
            print("TEXT:{}\n".format(msg.text))
            # Remove some non-English tweets
            en_msg = re.sub("[^\u0000-\u05C0\u2100-\u214F]+", "", msg.text)
            en_msg = re.sub(r"http\S+", "", en_msg)
            client_socket.send(en_msg.encode("utf-8"))
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
            self.disconnect()
            return False

    def on_error(self, status):
        print(status)
        return False

In [7]:
def send_data(c_socket, add_tags, remove_tags):
    """
    Send data to socket
    """
    global client_socket
    client_socket = c_socket
    stream = MyStream(BEARER_TOKEN)
    
    for tag in remove_tags:
        stream.delete_rules(tag)

    for tag in add_tags:
        stream.add_rules(tweepy.StreamRule(tag))
        
    print(stream.get_rules())

    stream.filter()

In [8]:
class TwitterClient:
    def __init__(self, TCP_IP, TCP_PORT):
        self.s = s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.s.bind((TCP_IP, TCP_PORT))

    def run_client(self, add_tags, remove_tags):
        try:
            self.s.listen(1)
            while True:
                print("Waiting for TCP connection...")
                conn, addr = self.s.accept()
                print("Connected... Starting getting tweets.")
                send_data(conn, add_tags, remove_tags)
                conn.close()
        except KeyboardInterrupt:
            exit()

In [ ]:
client = TwitterClient("localhost", 9001)
client.run_client(add_tags, remove_tags)

Waiting for TCP connection...
Connected... Starting getting tweets.
Response(data=[StreamRule(value='#worldcup2022', tag=None, id='1605095491011854336')], includes={}, errors=[], meta={'sent': '2022-12-21T02:11:56.739Z', 'result_count': 1})
Response(data=[StreamRule(value='#worldcup2022', tag=None, id='1605095491011854336')], includes={}, errors=[], meta={'sent': '2022-12-21T02:11:57.793Z', 'result_count': 1})
